In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 172ms/step - loss: 0.6730 - accuracy: 0.5674
Epoch 2/30
58/58 [==============================] - 10s 177ms/step - loss: 0.5413 - accuracy: 0.7338
Epoch 3/30
58/58 [==============================] - 10s 176ms/step - loss: 0.4769 - accuracy: 0.7941
Epoch 4/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4806 - accuracy: 0.7842
Epoch 5/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4851 - accuracy: 0.7744
Epoch 6/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4111 - accuracy: 0.8456
Epoch 7/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4030 - accuracy: 0.8313
Epoch 8/30
58/58 [==============================] - 10s 175ms/step - loss: 0.3720 - accuracy: 0.8488
Epoch 9/30
58/58 [==============================] - 10s 175ms/step - loss: 0.4576 - accuracy: 0.7897
Epoch 10/30
58/58 [==============================] - 10s 176ms/step - loss: 0.3561 - accura

58/58 [==============================] - 10s 166ms/step - loss: 0.3471 - accuracy: 0.8587
Epoch 22/30
58/58 [==============================] - 10s 165ms/step - loss: 0.3169 - accuracy: 0.8598
Epoch 23/30
58/58 [==============================] - 10s 165ms/step - loss: 0.2359 - accuracy: 0.9189
Epoch 24/30
58/58 [==============================] - 10s 166ms/step - loss: 0.2163 - accuracy: 0.9211
Epoch 25/30
58/58 [==============================] - 10s 166ms/step - loss: 0.1650 - accuracy: 0.9463
Epoch 26/30
58/58 [==============================] - 9s 163ms/step - loss: 0.1702 - accuracy: 0.9398
Epoch 27/30
58/58 [==============================] - 10s 165ms/step - loss: 0.1566 - accuracy: 0.9452
Epoch 28/30
58/58 [==============================] - 10s 166ms/step - loss: 0.2128 - accuracy: 0.9124
Epoch 29/30
58/58 [==============================] - 10s 165ms/step - loss: 0.2121 - accuracy: 0.9211
Epoch 30/30
58/58 [==============================] - 10s 164ms/step - loss: 0.1536 - accuracy: 

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 314ms/step - loss: 0.6684 - accuracy: 0.5761
Epoch 2/30
58/58 [==============================] - 18s 312ms/step - loss: 0.5238 - accuracy: 0.7558
Epoch 3/30
58/58 [==============================] - 18s 316ms/step - loss: 0.4916 - accuracy: 0.7974
Epoch 4/30
58/58 [==============================] - 18s 315ms/step - loss: 0.4320 - accuracy: 0.8313
Epoch 5/30
58/58 [==============================] - 18s 316ms/step - loss: 0.4712 - accuracy: 0.8149
Epoch 6/30
58/58 [==============================] - 18s 314ms/step - loss: 0.4218 - accuracy: 0.8368
Epoch 7/30
58/58 [==============================] - 18s 312ms/step - loss: 0.3239 - accuracy: 0.8729
Epoch 8/30
58/58 [==============================] - 18s 315ms/step - loss: 0.3101 - accuracy: 0.8795
Epoch 9/30
58/58 [==============================] - 18s 315ms/step - loss: 0.6084 - accuracy: 0.6495
Epoch 10/30
58/58 [==============================] - 18s 315ms/step - loss: 0.6548 - accura

58/58 [==============================] - 18s 311ms/step - loss: 0.5068 - accuracy: 0.7273
Epoch 22/30
58/58 [==============================] - 18s 314ms/step - loss: 0.4766 - accuracy: 0.7382
Epoch 23/30
58/58 [==============================] - 18s 314ms/step - loss: 0.4470 - accuracy: 0.7689
Epoch 24/30
58/58 [==============================] - 18s 315ms/step - loss: 0.4219 - accuracy: 0.7897
Epoch 25/30
58/58 [==============================] - 18s 313ms/step - loss: 0.4010 - accuracy: 0.8039
Epoch 26/30
58/58 [==============================] - 18s 314ms/step - loss: 0.3957 - accuracy: 0.8083
Epoch 27/30
58/58 [==============================] - 18s 312ms/step - loss: 0.3736 - accuracy: 0.8335
Epoch 28/30
58/58 [==============================] - 18s 315ms/step - loss: 0.3401 - accuracy: 0.8751
Epoch 29/30
58/58 [==============================] - 18s 314ms/step - loss: 0.3790 - accuracy: 0.8346
Epoch 30/30
58/58 [==============================] - 18s 313ms/step - loss: 0.3063 - accuracy: